## Daten aus API einlesen
Diesdasananas

Api Dokumentation gemäss: https://doc.iofrog.com/apidocumentation/api.html
Postman requests: https://www.getpostman.com/collections/eb80441a4500fd9a0634

In [22]:
# packages import
import requests
import json
import pandas as pd



Login to api

In [23]:
url = "https://www.iofrog.com/api/v2/auth/signin/"

payload = json.dumps({
  "username": "pascal.berger@students.fhnw.ch",
  "password": "123321"
})
headers = {
  'Content-Type': 'application/json',
  'Cookie': ''
}

login_response = requests.request("POST", url, headers=headers, data=payload)
#print(login_response.text)


In [24]:
login_data = pd.read_json(login_response.text)
access_token = login_data.loc['accessToken', 'token']
login_data

,user,token
firstName,Pascal,NaN
lastName,Berger,NaN
isActivated,True,NaN
level,expert,NaN
partner,heliot,NaN
lang,en,NaN
timeZone,Europe/Berlin,NaN
units,"{'temperature': 'C', 'system': 'M'}",NaN
accessToken,NaN,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6I...
refreshToken,NaN,605c54497746444305ca2511.93c2f327ec17d30bc4d67...


get devices data

In [25]:
url = "https://www.iofrog.com/api/v2/devices/dashboard"

payload = json.dumps([
  {
    "data": [],
    "count": 2,
    "meta": {}
  }
])
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json',
  'Cookie': ''
}

devices_response = requests.request("GET", url, headers=headers, data=payload)
#print(devices_response.text)


In [42]:
#print(devices_response.text)
#device_data = pd.read_json(devices_response.text)

device_dict = json.loads(devices_response.text)
device_data = pd.DataFrame.from_dict(device_dict['data'])
#device_data = pd.json_normalize(device_dict)
#device_dict['data']
device_data

,id,groupId,name,typeId,modeId,downlinkData,active,location,viewConfig,status,subscriptions,lastSourceBasedLocation,lastDataMessage,trackedKeys,badges
0,2203AAB,60476e95362d421c28ee0058,SimplePack POC_02203AAB TeamA,simple_pack_plus_full_3_0_6_0,reed_counter,0000000051410788,True,"{'lat': None, 'lng': None}","{'locationSource': 'fallback', 'view': 'last'}","{'isRegistered': True, 'isReadyForActivation':...","[{'_id': '60489bab362d421c28f49a02', 'provider...","{'geoplaces': [], '_id': '6059fbafb3ae30315afb...","{'_id': '605b57c4f4e46133dc9d069f', 'keys': [{...",[],[]
1,2203AF9,60476e95362d421c28ee0058,SimplePack POC_02203AF9 TeamA,simple_pack_plus_full_3_0_6_0,NaN,011307880F07500A000000000000515E,True,"{'lat': None, 'lng': None}","{'locationSource': 'atlas', 'view': 'last'}","{'isRegistered': True, 'isReadyForActivation':...","[{'_id': '6050c0467d354e09ee4aa73e', 'provider...",NaN,"{'_id': '605b445e7746444305bb202d', 'keys': [{...",[],[]
2,2203ACB,60476e95362d421c28ee0058,SimplePackPOC_02203ACB TeamB,simple_pack_plus_full_3_0_6_0,reed,0000000000000788,True,"{'lat': None, 'lng': None}","{'locationSource': 'fallback', 'view': 'last'}","{'isRegistered': True, 'isReadyForActivation':...","[{'_id': '60489995362d421c28f48edd', 'provider...","{'geoplaces': [], '_id': '605b68f2f4e46133dc9e...","{'_id': '605bc092f4e46133dca2fd8a', 'keys': [{...",[],[]
3,2203AFA,60476e95362d421c28ee0058,SimplePackPOC_02203AFA TeamB,simple_pack_plus_full_3_0_6_0,press_me,0000000000000788,True,"{'lat': None, 'lng': None}","{'locationSource': 'fallback', 'view': 'last'}","{'isRegistered': True, 'isReadyForActivation':...","[{'_id': '60489ad5362d421c28f49667', 'provider...","{'geoplaces': [], '_id': '605b4291f4e46133dc9b...","{'_id': '605c42ed7746444305c91bad', 'keys': [{...",[],[]
4,1F25047,60476e95362d421c28ee0058,SimplePackPTH_01F25047 TeamB\t,simple_pack_plus_full_3_0_6_0,put_me_back,,True,"{'lat': None, 'lng': None}","{'locationSource': 'fallback', 'view': 'last'}","{'isRegistered': True, 'isReadyForActivation':...","[{'_id': '60489b3f362d421c28f498b2', 'provider...","{'geoplaces': [], '_id': '6058b9d703380b4c4fd1...","{'_id': '605c4ba2f4e46133dca9fa98', 'keys': [{...",[],[]
5,1F2504E,60476e95362d421c28ee0058,SimplePackPTH_01F2504E TeamA,simple_pack_plus_full_3_0_6_0,NaN,00000F0707880106,True,"{'lat': None, 'lng': None}","{'locationSource': 'atlas', 'view': 'last'}","{'isRegistered': True, 'isReadyForActivation':...","[{'_id': '6050bd897d354e09ee4a5d37', 'provider...",NaN,"{'_id': '605c4da57746444305c9bc7d', 'keys': [{...",[],[]


In [45]:
url = "https://www.iofrog.com/api/v2/devices/2203AAB/messages/"

payload = json.dumps({
  "data": [
    {
      "_id": "1f704bc5-5055-43eb-a851-c87b6b0bf7b9",
      "isAlarm": False,
      "isDismissed": False,
      "timestamp": 1584093653,
      "type": "data_uplink",
      "seqNumber": 1,
      "keys": [
        {
          "isCode": False,
          "_id": "1111111111111111111111",
          "key": "key",
          "value": "value",
          "keyLocale": "Klíč",
          "valueLocale": "Hodnota",
          "color": "alarm",
          "icon": "icon name (from fontAwesome)"
        }
      ]
    }
  ],
  "count": 1
})
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json',
  'Cookie': ''
}

message_response = requests.request("GET", url, headers=headers, data=payload)

# print(response.text)


In [92]:
message_dict = json.loads(message_response.text)
#print(message_dict['data'])
message_data = pd.DataFrame.from_dict(message_dict['data'])

#print(message_data.loc[:,'keys'][0])
for index, keys in enumerate(message_data.loc[:,'keys']):
    # print(index)
    for key in keys:
        # print(key['value'])
        if key['key'] == 'temperature':
            message_data.loc[index, 'temperature'] = key['value'] + key['unit']
        if key['key'] == 'battery':
            message_data.loc[index, 'battery'] = key['value'] + key['unit']

message_data = message_data.drop(columns=['keys'])

message_data


,_id,timestamp,type,seqNumber,deviceId,created,__v,isAlarm,isDismissed,battery,temperature
0,605b57c4f4e46133dc9d069f,1616598979,data_bidir,38.0,2203AAB,2021-03-24T15:16:20.290Z,0,NaN,NaN,NaN,NaN
1,605b45387746444305bb2efb,1616594230,data_bidir,37.0,2203AAB,2021-03-24T13:57:12.136Z,0,NaN,NaN,NaN,NaN
2,605b45607746444305bb30d5,1616594230,service_ack,NaN,2203AAB,2021-03-24T13:57:52.781Z,0,NaN,NaN,NaN,NaN
3,605b3449f4e46133dc9a8e96,1616589897,data_bidir,36.0,2203AAB,2021-03-24T12:44:57.988Z,0,NaN,NaN,3V,20.5°C
4,605a60aa82c5542ccc8663db,1616535721,data_bidir,35.0,2203AAB,2021-03-23T21:42:02.436Z,0,NaN,NaN,3V,19°C
5,605a023eb3ae30315afc75b5,1616511548,data_bidir,34.0,2203AAB,2021-03-23T14:59:10.293Z,0,NaN,NaN,NaN,NaN
6,605a025c82c5542ccc80d88f,1616511548,service_ack,NaN,2203AAB,2021-03-23T14:59:40.564Z,0,NaN,NaN,NaN,NaN
7,605a01f982c5542ccc80cf55,1616511480,data_bidir,32.0,2203AAB,2021-03-23T14:58:01.691Z,0,NaN,NaN,NaN,NaN
8,605a000f82c5542ccc80b096,1616510989,data_bidir,31.0,2203AAB,2021-03-23T14:49:51.314Z,0,NaN,NaN,NaN,NaN
9,605a0010b3ae30315afc4d76,1616510983,data_bidir,30.0,2203AAB,2021-03-23T14:49:52.374Z,0,NaN,NaN,NaN,NaN
